In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import scipy.stats as ss
from sklearn.preprocessing import OrdinalEncoder

In [3]:
data = pd.read_stata('PS1/oil.dta')
data.head()

,okpo,companyname,year,fixedassets,sales,profit,profitaftertax,employees,wages,postcode,region,okved2008
0,75775799,rekaveri,2008.0,5.684062,0.000000,0.000000,0.000000,2.0,NaN,169710,Komi Republic,111011
1,75775799,rekaveri,2009.0,0.264514,0.000000,0.000000,-484.522644,2.0,NaN,169710,Komi Republic,111011
2,75775799,rekaveri,2010.0,0.000000,0.000000,0.000000,-418.413879,2.0,NaN,169710,Komi Republic,111011
3,74066994,unistroi,2005.0,0.382210,18.276581,18.276581,0.451703,15.0,NaN,103055,Moscow,111000
4,73614678,interneft,2007.0,0.000000,0.000000,0.000000,0.000000,NaN,NaN,460000,Orenburg Region,111011


**Simple Static Cobb-Douglas**

$$
Q_j = M_jK_j^\alpha L_j^\beta\\
m_j = \mu + e_j\\
q_j = \mu + \alpha k_j + \beta l_j + e_j\\
\theta = \begin{bmatrix} \mu \\ \alpha \\ \beta \end{bmatrix}, X = \begin{bmatrix}
1 \\ k_j \\
l_j \end{bmatrix}\\
\hat{\theta} = (X'X)^{-1} (X' y)
$$

**Returns to Scale**

$$
Q_j(K_j, L_j) = M_j K_j^\alpha L_j^\beta \\
Q_j(\gamma K_j, \gamma L_j) = \gamma^{\alpha + \beta} M_j K_j^\alpha L_j^\beta = \gamma^{\alpha + \beta} Q_j(K_j, L_j)
$$
Function satisfies:
1. CRS if $\gamma Q_j(K_j, L_j) = Q_j(\gamma K_j, \gamma L_j) \iff \alpha + \beta = 1$
2. DRS if $\gamma Q_j(K_j, L_j) > Q_j(\gamma K_j, \gamma L_j) \iff \alpha + \beta < 1$
3. IRS if $\gamma Q_j(K_j, L_j) < Q_j(\gamma K_j, \gamma L_j) \iff \alpha + \beta > 1$

**Testing CRS**

Null Hypothesis is $$H_0: \alpha + \beta - 1 = 0$$

We can use the *Wald Statistic* under linear hypothesis:

$$
H_0: R'\theta = \theta_0 \\
H_1: R'\theta \neq \theta_0\\
\theta' = \begin{bmatrix} \mu &  \alpha & \beta \end{bmatrix} \\
R' = \begin{bmatrix}
0 & 1 & 1
\end{bmatrix}\\
\theta_0 = 1
$$
Wald statistic is 
$$
W = (R'\hat{\theta} - \theta_0)'\left(R'\hat{V}_{\hat{\theta}}R\right)^{-1}(R'\hat{\theta} - \theta_0)\\
W \overset{d}{\to} \chi^2_q, q = 1\\
\text{P-value} = 1 - \chi^2_1(W)
$$

**Testing IRS**

Null Hypothesis is
$$
H_0: \alpha + \beta > 1\\
H_1: \alpha + \beta \leq 1
$$




In [9]:
data['q'] = np.log(data['sales'] + 0.000001)
data['k'] = np.log(data['fixedassets'] + 0.000001)
data['l'] = np.log(data['employees'] + 0.000001)
data['const'] = 1

/usr/local/anaconda3/envs/tensorflow_on_mac/lib/python3.8/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
oil = data.loc[(data.q.isna() == False) & (data.l.isna() == False) & (data.l.isna() == False), ['okpo', 'year', 'q', 'const', 'k', 'l']]
oil.head()

const = oil.const
l = oil.l
k = oil.k
y = oil.q

X = np.stack((const, k,l), axis = 1)

In [11]:
class OLS:
    def __init__(self, y, X, round = False, FE = False, id = None):
        self.y, self.X = y, X
        self.N_COEF = self.X.shape[1]
        if FE:
            self.X = np.append(self.X, pd.get_dummies(id, drop_first = True), axis = 1)

        self.BETA = self.BETA()
        self.VCOV = self.VCOV(self.BETA)
        self.SE = np.sqrt(self.VCOV.diagonal())
        self.res = dict(zip(np.round(self.BETA,4), np.round(self.SE,4)))
        self.Wald_CRS = self.Wald_CRS(self.BETA, self.VCOV)
        if FE:
            self.BETA_FE = self.BETA[(self.N_COEF+1):]
            self.BETA = self.BETA[:self.N_COEF]
            self.SE_FE = self.SE[(self.N_COEF+1):]
            self.SE = self.SE[:self.N_COEF]
            self.res = dict(zip(np.round(self.BETA,4), np.round(self.SE,4)))
        
    def BETA(self):
        X, y = self.X, self.y
        BETA = np.linalg.inv(X.T @ X) @ X.T @ y
        return BETA 
    
    def VCOV(self, BETA):
        X, y = self.X, self.y 
        res = y - X @ BETA
        s2 = (1/(X.shape[0]-X.shape[1]))*sum(res**2) 
        VCOV = s2*np.linalg.inv(X.T @ X)
        return VCOV
    
    def Wald_CRS(self, BETA, VCOV):
        B0 = 1
        R = np.array(np.zeros_like(BETA), ndmin=2).T
        R[1:2] = 1
        W = (R.T @ BETA - B0).T @ np.linalg.inv(R.T @ VCOV @ R) @ (R.T @ BETA - B0)
        PVAL = 1 - ss.chi2.cdf(W, df = 1)
        return (W, PVAL)

In [13]:
m_OLS = OLS(y, X)
res = pd.DataFrame(m_OLS.res, index = [0]).T
res['est'] = res.index
res.index = ['const', 'k', 'l']
res = res.loc[:, ['est', 0]]
res.columns = ['est', 'se']
print(res.style.to_latex())

\begin{tabular}{lrr}
 & est & se \\
const & -8.452300 & 0.176100 \\
k & 0.230300 & 0.013200 \\
l & 2.768600 & 0.049100 \\
\end{tabular}



In [14]:
m_FE = OLS(y, X, FE = True, id = oil.okpo)
res = pd.DataFrame(m_FE.res, index = [0]).T
res['est'] = res.index
res.index = ['const', 'k', 'l']
res = res.loc[:, ['est', 0]]
res.columns = ['est', 'se']
print(res.style.to_latex())

\begin{tabular}{lrr}
 & est & se \\
const & -4.864500 & 3.134300 \\
k & 0.187300 & 0.023100 \\
l & 2.198600 & 0.096900 \\
\end{tabular}



## CRS and IRS

In [15]:
m_OLS.Wald_CRS

(3416.2314975607514, 0.0)

In [17]:
VCOV_alpha = m_OLS.VCOV[1,1]
VCOV_beta = m_OLS.VCOV[2,2]
COV_alphabeta = m_OLS.VCOV[1,2]
se_alpha_p_beta = np.sqrt(VCOV_alpha + VCOV_beta + COV_alphabeta)
alpha = m_OLS.BETA[1]
beta = m_OLS.BETA[2]
t = (alpha+beta)/se_alpha_p_beta
print(t)

63.64301644216705


## Time Trend

In [19]:
# time trend
oil['time'] = oil.year - min(oil.year)
oil['time_x_k'] = oil.time * oil.k
oil['time_x_l'] = oil.time * oil.l

X1 = np.array(oil.loc[:, ['const', 'k', 'l', 'time', 'time_x_k', 'time_x_l']])
m1_OLS = OLS(y, X1)
m1_FE = OLS(y, X1, FE = True, id = oil.okpo)
res = pd.DataFrame(m1_OLS.res, index = [0]).T
res['est'] = res.index
res.index = ['const', 'k', 'l', 'time', 'time_x_k', 'time_x_l']
res = res.loc[:, ['est', 0]]
res.columns = ['est', 'se']
print(res.style.to_latex())

\begin{tabular}{lrr}
 & est & se \\
const & -9.638700 & 0.352400 \\
k & 0.134400 & 0.027300 \\
l & 3.247900 & 0.099400 \\
time & 0.288300 & 0.076600 \\
time_x_k & 0.023100 & 0.005700 \\
time_x_l & -0.114800 & 0.020800 \\
\end{tabular}



In [20]:
res = pd.DataFrame(m1_FE.res, index = [0]).T
res['est'] = res.index
res.index = ['const', 'k', 'l', 'time', 'time_x_k', 'time_x_l']
res = res.loc[:, ['est', 0]]
res.columns = ['est', 'se']
print(res.style.to_latex())

\begin{tabular}{lrr}
 & est & se \\
const & -4.562300 & 3.141100 \\
k & 0.049200 & 0.034000 \\
l & 2.496200 & 0.118600 \\
time & 0.097200 & 0.068500 \\
time_x_k & 0.031200 & 0.005700 \\
time_x_l & -0.075300 & 0.017100 \\
\end{tabular}



In [21]:
oil['crisis'] = 0
oil.loc[(oil.year == 2008) | (oil.year == 2009), 'crisis'] = 1

X2 = np.array(oil.loc[:, ['const', 'k', 'l', 'time', 'time_x_k', 'time_x_l', 'crisis']])

m2_OLS = OLS(y, X2)
m2_FE = OLS(y, X2, FE = True, id = oil.okpo)


res = pd.DataFrame(m2_OLS.res, index = [0]).T
res['est'] = res.index
res.index = ['const', 'k', 'l', 'time', 'time_x_k', 'time_x_l', 'crisis']
res = res.loc[:, ['est', 0]]
res.columns = ['est', 'se']
print(res.style.to_latex())

\begin{tabular}{lrr}
 & est & se \\
const & -8.816000 & 0.365300 \\
k & 0.136900 & 0.027100 \\
l & 3.217700 & 0.098800 \\
time & 0.232800 & 0.076400 \\
time_x_k & 0.022100 & 0.005700 \\
time_x_l & -0.114000 & 0.020700 \\
crisis & -1.702100 & 0.215600 \\
\end{tabular}



In [22]:
res = pd.DataFrame(m2_FE.res, index = [0]).T
res['est'] = res.index
res.index = ['const', 'k', 'l', 'time', 'time_x_k', 'time_x_l', 'crisis']
res = res.loc[:, ['est', 0]]
res.columns = ['est', 'se']
print(res.style.to_latex())

\begin{tabular}{lrr}
 & est & se \\
const & -4.611300 & 3.136100 \\
k & 0.051400 & 0.034000 \\
l & 2.500800 & 0.118400 \\
time & 0.077100 & 0.068700 \\
time_x_k & 0.030900 & 0.005700 \\
time_x_l & -0.075900 & 0.017100 \\
crisis & -0.552100 & 0.157200 \\
\end{tabular}



## Olley-Pakes Procedure

1. Add investment variable
2. Add polynomial expansion of $g(k_{jt}, i_{jt})$. That is 
$$
\gamma_0 + \gamma_1 k + \gamma_2 i + \gamma_3 k^2 + \gamma_4 i^2 + \gamma_5 i\cdot k
$$
2. Estimate 
    $$q_{jt} = \beta l_{jt} + \alpha k_{jt} + g(k_{jt}, i_{jt}) + e_{jt}$$ 
    to obtain 
    $$
    \widehat{\beta}, \widehat{\phi(k_{jt}, i_{jt})} \equiv  g(k_{jt}, i_{jt}) + \mu +   \alpha k_{jt}$$ 
    Note that under polynomial expansion 
    $$
    \widehat{\phi(k_{jt}, i_{jt})} = \gamma_0 + \gamma_1 k + \gamma_2 i + \gamma_3 k^2 + \gamma_4 i^2 + \gamma_5 i\cdot k
    $$



In [23]:
oil['next_k'] = oil.groupby('okpo')['k'].shift(-1)
oil['prev_k'] = oil.groupby('okpo')['k'].shift(1)
oil['i'] = oil['next_k'] - oil['k']
oil.loc[oil.i < 0, 'i'] = 0
ord_enc = OrdinalEncoder()
oil["id"] = ord_enc.fit_transform(oil[["okpo"]])

In [24]:
k = np.array(oil.k)
l = np.array(oil.l)
y = np.array(oil.q)
i = np.array(oil.i)
prev_k = np.array(oil.prev_k)
id = np.array(oil.id)
year = np.array(oil.year)

k2, i2, ki = k**2, i**2, k*i
const = np.zeros_like(k) + 1

In [25]:
class OlleyPakes:
    def __init__(self, y, k, l, i, id, year):
        k2, i2, ki = k**2, i**2, k*i
        const = np.zeros_like(k) + 1
        
        '''
        Delete NAs
        '''
        DAT = np.stack((y, l, const, k, i, k2, i2, ki, id, year), axis = 1)
        DAT = self.dropna(DAT)
        self.DAT = DAT
        self.l = self.DAT[:, 1]
        self.id, self.year = DAT[:, 8], DAT[:, 9] 
        self.y1, self.X1 = DAT[:, 0], DAT[:, 1:7]

        '''
        Results from 1st stage
        '''
        self.beta, self.phi, self.VCOV1 = self.step1(self.X1, self.y1)
        self.beta_se = np.sqrt(self.VCOV1[0,0])
        
        '''
        Preparing 2nd stage
        '''
        # shifting k and phi and droping nas
        self.k = DAT[:, 3]
        self.const = np.zeros_like(self.k) + 1
        self.y2 = self.y1 - self.beta * self.l 
        
        DAT2 = pd.DataFrame(np.stack((self.y2, self.k, self.const, self.phi, self.id, self.year),axis = 1))
        DAT2.columns = ['y - bl', 'k', 'const', 'phi', 'id', 'year']
        DAT2['prev_k'] = DAT2.groupby('id')['k'].shift(1)
        DAT2['prev_phi'] = DAT2.groupby('id')['phi'].shift(1)
        DAT2 = DAT2.dropna()

        # main coefficient 
        y2, k = np.array(DAT2['y - bl']), np.array(DAT2.k)
        
        # polynomial expansion
        const = np.zeros_like(k) + 1
        prev_phi, prev_k = np.array(DAT2.prev_phi), np.array(DAT2.prev_k)
        prev_k2, prev_phi2, prev_phi_k = prev_k**2, prev_phi**2, prev_k * prev_phi        
        
        # final frame
        X2 = np.stack((k, const, prev_k, prev_phi, prev_k2, prev_phi2, prev_phi_k), axis = 1)
        
        '''
        Run 2ns stage
        '''
        COEF2 = np.linalg.inv(X2.T @ X2) @ X2.T @ y2
        
        res = y2 - X2 @ COEF2
        s2 = (1/(X2.shape[0]-X2.shape[1]))*sum(res**2) 
        VCOV2 = s2*np.linalg.inv(X2.T @ X2)        
        
        self.alpha = COEF2[0]
        self.alpha_se = np.sqrt(VCOV2[0,0])
        est = np.round([self.alpha, self.beta],4)
        se = np.round([self.alpha_se, self.beta_se],4)
        self.res = dict(zip(est, se))
        

    def dropna(self,a):
        return a[~np.isnan(a).any(axis=1), :]


    def step1(self, X, y):
        '''
        estimate and obtain prev_phi
        '''
        COEF = np.linalg.inv(X.T @ X) @ X.T @ y
        beta = COEF[0]
        phi_coef = COEF[1:]
        phi_var = X[:, 1:]
        phi = phi_var @ phi_coef
        res = y - X @ COEF
        s2 = (1/(X.shape[0]-X.shape[1]))*sum(res**2) 
        VCOV = s2*np.linalg.inv(X.T @ X)
        return beta, phi, VCOV

In [26]:
y = np.array(oil.q)
k = np.array(oil.k)
l = np.array(oil.l)
i = np.array(oil.i)
prev_k = np.array(oil.prev_k)
okpo = np.array(oil.okpo)
year = np.array(oil.year)

In [27]:
OP = OlleyPakes(y, k, l, i, id, year)
OP.res

{0.1715: 0.0449, 2.6689: 0.0614}